# 知识区域：
1. apply 函数的用法； 
    * 作用于字典dic.items()，列表，元组等可迭代的数据结构
    * 效果： 其中每个元素为自变量，apply的函数
    * 例子如下：
    
    ``` datalist  = list(data['text'])
    cutfun = lambda x: list(jieba.cut(x))
    senctences = data['text'].apply(cutfun)
2. jieba 的用法：
```


In [29]:


from gensim.corpora import dictionary
from gensim import corpora,models,similarities

import pandas as pd
import numpy as np
import re
import pickle
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile, common_texts
from keras import Sequential
from keras import regularizers
from keras.preprocessing import sequence
import keras
import jieba


In [30]:
#loas data
data = pd.read_csv("sohu.csv")
labels = data['label']
data.head()

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...


In [31]:
labeltag=list(labels.unique())
print(labeltag)
for i in range(len(labels)):
    for j in range(len(labeltag)):
        if(labels[i]==labeltag[j]):
            labels[i]=j
            break
labels[:5]

['pic', 'news', 'sports', 'business', 'caipiao', 'yule', 'mil', 'cul']


0    0
1    1
2    1
3    1
4    1
Name: label, dtype: object

In [32]:
# 首先用jieba 分词， 然后生成list of wordlist 
# 然后将它放到 word2vecor 里面

In [33]:
# 分词
datalist = list(data['text'])
length = len(datalist)
# 分词
# 定义分词函数
sentences = []
for i in range(length):
    split_text = list(jieba.cut(datalist[i]))
    sentences.append(split_text)# list 的list 
sentences[:2]

[['高清',
  '：',
  '彭帅',
  '谢淑薇',
  '遗憾',
  '遭',
  '逆转',
  '出局',
  ' ',
  '击掌',
  '互相',
  '鼓励',
  '责编',
  ':',
  '马克',
  '杰',
  ' ',
  '日期',
  ':',
  '2013',
  '-',
  '10',
  '-',
  '4',
  ' ',
  '彭帅',
  '谢淑薇',
  '互相',
  '鼓励',
  ' ',
  '北京',
  '时间',
  '10',
  '月',
  '4',
  '日',
  '消息',
  ',',
  '2013',
  '年',
  '中国',
  '网球',
  '公开赛',
  '结束',
  '了',
  '一场',
  '女双',
  '半决赛',
  '的',
  '争夺',
  ',',
  '赛会',
  '2',
  '号',
  '种子',
  '―',
  '―',
  '我国',
  '金花',
  '彭帅',
  '和',
  '中华',
  '台北',
  '名将',
  '谢淑薇',
  '的',
  '海峡',
  '组合',
  '错过',
  '了',
  '先胜',
  '一盘',
  '、',
  '抢',
  '十局',
  '8',
  '-',
  '4',
  '领先',
  '的',
  '优势',
  ',',
  '连',
  '丢',
  '六分',
  '之后',
  '以',
  '6',
  '-',
  '3',
  '/',
  '1',
  '-',
  '6',
  '/',
  '8',
  '-',
  '10',
  '惨遭',
  '逆转',
  ',',
  '无缘',
  '女双',
  '决赛',
  '。',
  '(',
  '搜狐',
  '-',
  '李志岩',
  '/',
  '摄',
  ')'],
 ['河南',
  '周口',
  '路边',
  '秸秆',
  '燃烧',
  ' ',
  '2',
  '岁',
  '幼儿',
  '被',
  '熏',
  '晕',
  '(',
  '图',
  ')',
  '\xa0',
  ' ',
  '10',
  '月',
  '3

In [34]:
# 训练Word2vec模型

model = Word2Vec(
                 size=100,  # 词向量维度
                 min_count=5,  # 词频阈值
                 window=5)  # 窗口大小
model.build_vocab(sentences)
model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)
model.save('souhu.model')# 保存模型


D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  
D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
modelw = Word2Vec.load('kesic2.model')# 加载模型
print(len(modelw.wv.vocab))

D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


36572


In [36]:
gensim_dict = dictionary.Dictionary()

gensim_dict.doc2bow(model.wv.vocab.keys(), allow_update=True)
w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # K ：索引，V： vactary，词语的索引，从1开始编号 print(w2indx)
w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
#corpora.MmCorpus.serialize('textmatrix.mm', corpus) # 存入硬盘，以备后需
# gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# 存起来
output = open("souhu.pkl", 'wb')
pickle.dump(w2indx, output)  # 索引字典
pickle.dump(w2vec, output)  # 词向量字典
output.close()

D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [37]:
# 读取大语料文本
f = open("souhu.pkl", 'rb')  # 预先训练好的
index_dict = pickle.load(f)  # 索引字典，{单词: 索引数字}
word_vectors = pickle.load(f)  # 词向量, {单词: 词向量(100维长的数组)}
# word_vectors.values().shape

new_dic = index_dict
print(len(new_dic))
word_length = word_vectors['19'].shape[0]
word_length

36572


100

In [21]:
# 创建权重矩阵
print (u"Setting up Arrays for Keras Embedding Layer...")
n_symbols = len(index_dict) + 1  # 索引数字的个数，因为有的词语索引为0，所以+1
#embedding_weights = np.zeros((n_symbols,word_length))  # 创建一个n_symbols * word_length的0矩阵
embedding_weights = ((np.random.rand(n_symbols,word_length)) - 0.5) / 5.0  # 创建一个随机初始化一个 n_symbols * word_length的矩阵
for w, index in index_dict.items():  # 从索引为1的词语开始，用词向量填充矩阵
    embedding_weights[index, :] = word_vectors[w]  # 词向量矩阵，第一行是0向量（没有索引为0的词语，未被填充）
# embedding_weights[2, :]

Setting up Arrays for Keras Embedding Layer...


In [22]:

X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(sentences, labels, test_size=0.2)

In [23]:
def text_to_index_array(p_new_dic, p_sen):  # 文本转为索引数字模式
    new_sentences = []
    for sen in p_sen:
        new_sen = []
        for word in sen:
            try:
                new_sen.append(p_new_dic[word])  # 单词转索引数字
            except:
                new_sen.append(0)  # 索引字典里没有的词转为数字0
        new_sentences.append(new_sen)

    return np.array(new_sentences)


In [38]:
# 转为数字索引形式

X_train = text_to_index_array(new_dic, X_train_l)

X_test = text_to_index_array(new_dic, X_test_l)
print( u"训练集shape： ", X_train.shape)
print(u"测试集shape： ", X_test.shape)
y_train = np.array(y_train_l)  # 转numpy数组
y_test = np.array(y_test_l)
print(y_train.shape)

训练集shape：  (4736,)
测试集shape：  (1185,)
(4736,)


In [25]:
# 参数设置
vocab_dim = 100  # 向量维度
maxlen = 140  # 文本保留的最大长度
batch_size = 32
n_epoch = 100
input_length = 140

In [26]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
earlystop_lr = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,min_delta=0.0001)

In [39]:
def train_lstm(p_n_symbols, p_embedding_weights, p_X_train, p_y_train, p_X_test, p_y_test):
    print (u'创建模型...')
    model = Sequential()
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=p_n_symbols,
                        mask_zero=True,
                        weights=[p_embedding_weights],
                        input_length=input_length))

    model.add(LSTM(output_dim=50,
                   activation='sigmoid',
                   inner_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    print (u'编译模型...')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print (u"训练...")
    print(model.summary())
    model.fit(p_X_train, p_y_train, batch_size=batch_size, nb_epoch=n_epoch,
              validation_data=(p_X_test, p_y_test),
             callbacks=[reduce_lr,earlystop_lr])
    
    print (u"评估...")
    score, acc = model.evaluate(p_X_test, p_y_test, batch_size=batch_size)
    print ('Test score:', score)
    print ('Test accuracy:', acc)
    return model



In [ ]:
# 将句子截取相同的长度maxlen，不够的补0

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

models = train_lstm(n_symbols, embedding_weights, X_train, y_train, X_test, y_test)

Pad sequences (samples x time)
X_train shape: (4736, 140)
X_test shape: (1185, 140)
创建模型...


D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", units=50, recurrent_activation="hard_sigmoid")`
  if sys.path[0] == '':


编译模型...
训练...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 140, 100)          3657300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 3,687,551
Trainable params: 3,687,551
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.


D:\1installs\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 4736 samples, validate on 1185 samples
Epoch 1/100
4736/4736 [==============================] - 72s 15ms/step - loss: -4.1414 - acc: 0.4996 - val_loss: -6.8313 - val_acc: 0.4878
Epoch 2/100
4736/4736 [==============================] - 76s 16ms/step - loss: -8.0402 - acc: 0.5091 - val_loss: -10.2437 - val_acc: 0.4878
Epoch 3/100
4736/4736 [==============================] - 69s 15ms/step - loss: -10.7233 - acc: 0.5091 - val_loss: -12.7043 - val_acc: 0.4878
Epoch 4/100
4736/4736 [==============================] - 75s 16ms/step - loss: -11.6699 - acc: 0.5091 - val_loss: -12.8873 - val_acc: 0.4878
Epoch 5/100
4736/4736 [==============================] - 72s 15ms/step - loss: -11.9882 - acc: 0.5091 - val_loss: -12.8884 - val_acc: 0.4878
Epoch 6/100
4736/4736 [==============================] - 71s 15ms/step - loss: -12.0745 - acc: 0.5091 - val_loss: -12.8879 - val_acc: 0.4878
Epoch 7/100
4736/4736 [==============================] - 84s 18ms/step - loss: -12.1457 - acc: 0.5091 - val_l